In [117]:
from html.parser import HTMLParser
import pandas as pd

In [113]:
class BNCParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.parsed_instances = list()
        self.recording = False
        self.this_instance = dict()
        self.this_data = list()
        
    def handle_starttag(self, tag, attrs):
        if tag == 'instance':
            self.this_instance['id'] = attrs[0][1]
        if tag == 'context':
            self.recording =  True
    
    def handle_startendtag(self, tag, attrs):
        if tag == 'answer':
            self.this_instance['sense'] = attrs[1][1].split('%')[1]
            
    def handle_data(self, data):
        if self.recording:
            self.this_data.append(data)
            
    def handle_endtag(self, tag):
        if tag == 'context':
            self.recording = False
        if tag == 'instance':
            self.this_data = ' '.join(self.this_data)
            self.this_instance['data'] = self.this_data.strip()
            self.parsed_instances.append(self.this_instance)

            self.this_instance = dict()
            self.this_data = list()

In [119]:
filename = 'plant-1.wsd'

In [127]:
parser = BNCParser()

with open(filename, 'r') as f:
    parser.feed(f.read())

In [128]:
df = pd.DataFrame(parser.parsed_instances)

In [129]:
df.tail()

,id,sense,data
183,plant.1000195,living,Above jacket detail from The Glasshouse Garden...
184,plant.1000197,living,(3) THE POWER OF EVERYDAY LANGUAGE The science...
185,plant.1000198,living,Here Minton would occasionally stretch out und...
186,plant.1000199,living,This enabled the preservation of live programm...
187,plant.1000200,living,Nothing worked on that one. Everything went wr...


In [133]:
df.shape

(188, 3)